In [1]:
%load_ext dotenv
%dotenv

In [2]:
from sklearn.pipeline import FeatureUnion, Pipeline
from pipline import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from gensim.models import CoherenceModel
import gensim

In [6]:
clearing = Pipeline([
    ('vk_group_post', GetAllTextPostsFromGroupPipeline(max_repeat_count=0)),
    ('clear_text', ClearTextPipeline()),
    ('normalization', TokenNormalizationPipeline()),
])

tf = clearing.fit_transform("https://vk.com/maiuniversity")

In [ ]:
clearing = Pipeline([
    ('wall_photos', GetUserGroupsPipeline()),
])
